In [0]:
db = "deltadb"
 
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")
 
spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

In [0]:
import random
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
path_data = "dbfs:/FileStore/tables/data.csv"

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/")
#dbutils.fs.rm("dbfs:/FileStore/tables/data.csv")

In [0]:
df = spark.read.csv(path_data, header=True, inferSchema=True)

In [0]:
df.show(7)

In [0]:
df.printSchema()

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("sales")

In [0]:
display(spark.sql("select count(*) from sales"))

In [0]:
display(spark.sql("select * from sales"))

In [0]:
%sql
SELECT s.region, s.manager, round(avg(s.val),1) as avg_val
FROM sales as s
GROUP BY s.region, s.manager
ORDER BY s.region, s.manager

In [0]:
%sql DESCRIBE HISTORY sales

In [0]:
new_column = [StructField("new_val", IntegerType(), True)]
new_schema = StructType(spark.table("sales").schema.fields + new_column)
data = [(200, datetime.now(), "r8","m11","pr25", 55, 100),
        (201, datetime.now(), "r8","m11","pr25", 75, 101)]
 
new_data = spark.createDataFrame(data, new_schema)
new_data.printSchema()

In [0]:
new_data.write.format("delta").mode("append").saveAsTable("sales")

In [0]:
new_data.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("sales")

In [0]:
%sql SELECT s.* FROM sales as s WHERE s.key_id IN (200,201)

In [0]:
%sql
DESCRIBE HISTORY sales

In [0]:
spark.sql("SELECT COUNT(*) FROM sales").show()
spark.sql("SELECT COUNT(*) FROM sales VERSION AS OF 0").show()

In [0]:
%sql RESTORE sales VERSION AS OF 0

In [0]:
spark.sql("SELECT COUNT(*) FROM sales").show()

In [0]:
%sql
DELETE FROM sales WHERE key_id=199;
SELECT * FROM sales WHERE key_id=199

In [0]:
%sql
INSERT INTO sales
SELECT * FROM sales VERSION AS OF 0
WHERE key_id=199

In [0]:
%sql SELECT * FROM sales WHERE key_id=199

In [0]:
%sql UPDATE sales SET val = 1000 WHERE key_id = 199

In [0]:
%sql SELECT * FROM sales WHERE key_id = 199

In [0]:
data = [(199, datetime.now(), "r6","m10","pr47", 5000),  # record to update
        (200, datetime.now(), "r6","m10","pr47", 5001)]  # record to insert
data_schema = spark.table("sales").schema
spark.createDataFrame(data, data_schema).createOrReplaceTempView("merge_table")
spark.sql("SELECT * FROM merge_table").show()

In [0]:
%sql
MERGE INTO sales AS s
USING merge_table AS m
ON s.key_id = m.key_id
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *;

In [0]:
%sql SELECT * FROM sales WHERE key_id IN (199, 200)

In [0]:
%sql
VACUUM sales